## Solving Linear Programs using Interior Point Methods

The main idea of the primal-dual method is to solve this set of non-linear equations for a decreasing set of $\mu$. A solution to the equations when $\mu = 0$ is a complementary solution.

\begin{align}
    Ax - b\tau &= 0 \\
    A^T y + z - c\tau &= 0 \\
    -c^Tx + b^Ty - \kappa  &= 0 \\
    Xz - \mu \textbf{e} &= 0 \\
    \tau \kappa - \mu &= 0 \\
\end{align}

Where:
\begin{align}
    x \in \mathbb{R} ^n & \succcurlyeq 0 \\
    z \in \mathbb{R} ^n & \succcurlyeq 0 \\
    \tau \in \mathbb{R} & \geq 0 \\
    \kappa \in \mathbb{R} & \geq 0 \\
    \mu \in \mathbb{R} & \geq 0 \\
    y \in \mathbb{R} ^m & \quad \text{free} \\ 
    A &: \mathbb{R}^n \rightarrow \mathbb{R} ^m \quad \text{A is an $m$ x $n$ matrix}\\
    b &: \mathbb{R}^m \\
    c &: \mathbb{R}^n \\
    x &: \mathbb{R}^n \quad \text{the decision variable we are optimizing for} \\
    X &: \mathbb{R}^n \rightarrow \mathbb{R} ^n \quad \text{X is an $n$ x $n$ diagonal matrix with $x_i$ on the diagonal } \\
    \textbf{e} \in \mathbb{R}^n&: \text{A column vector of 1's} \\
\end{align}


In [1]:
using LinearAlgebra
using Statistics
using Formatting

In [2]:
function get_solver(M)
    (r) -> M \ r
end

get_solver (generic function with 1 method)

In [3]:
function initial_start(m, n)
    x₀ = ones(n)
    y₀ = zeros(m)
    z₀ = ones(n)
    τ₀ = 1
    κ₀ = 1
    return x₀, y₀, z₀, τ₀, κ₀
end

initial_start (generic function with 1 method)

In [4]:
η₁(γ) = (1 - γ)

η₁ (generic function with 1 method)

In [5]:
β = 0.1
get_gamma(z, x) = β * mean(z .* x)

get_gamma (generic function with 1 method)

In [6]:
function get_step(x, dx, z, dz, τ, dtau, κ, dkappa, α₀)
    ix = dx .< 0
    iz = dz .< 0
    
    αx     = α₀ * (any(ix) ? minimum(x[ix] ./ -dx[ix]) : 1)
    αtau   = α₀ * (dtau < 0 ? τ / -dtau : 1)
    αz     = α₀ * (any(iz) ? minimum(z[iz] ./ -dz[iz]) : 1)
    αkappa = α₀ * (dkappa < 0 ? κ / -dkappa : 1)
    
    α = min(1, αx, αtau, αz, αkappa)
    return α
end

get_step (generic function with 1 method)

Assuming an initial solution $(x, \tau, y, z, \kappa)$ where $(x, \tau, z, \kappa) > 0$, then search direction is defined by the following set of linear equations.

\begin{align}
    Ad_x  - bd_{\tau} &= \eta r_p \\
    A^Td_y + d_z - cd_{\tau} &= \eta r_d \\
    -c^Tx + b^Ty - \kappa &= 0 \\
    Xz &= \mu \textbf{e} \\
    \tau \kappa &= \mu \\
\end{align}

and:
\begin{align}
    Zd_x + Xd_z &= -Xz + \gamma \mu \textbf{e} \\
    \kappa d_{\tau} + \tau d_{\kappa} &= -\tau \kappa + \gamma \mu \\
\end{align}

where:
\begin{align}
    r_p &:= b \tau - Ax = -(Ax - b \tau) \\
    r_d &:= c \tau - A^Ty - z = - (A^Ty + z - c \tau)\\
    r_g &:= \kappa + c^Tx - b^Ty = - (b^Ty - c^Tx - \kappa) \\
    \mu &:= \begin{bmatrix} x \\ \tau \end{bmatrix}^T \begin{bmatrix} z \\ \kappa \end{bmatrix}  \div (n+1)\\
\end{align}

$\gamma$ and $\eta$ are two nonnegative parameters. If $\gamma = \eta = 1$ the search direction defined above is equivalent to one newton step.

In each iteration, the Newton equation system below is solved:
 
$$\begin{bmatrix} A & -b &       &     &   &      \\  
                      & -c     & A^T & I &      \\ 
                 -c^T &        & b^T &   & -1   \\
                 Z    &        &     & X &      \\
                      & \kappa &     &   & \tau
\end{bmatrix} \begin{bmatrix}
    d_x \\
    d_{\tau} \\
    d_y \\
    d_z \\
    d_{\kappa}
\end{bmatrix}  = \begin{bmatrix}
    \hat r_p \\
    \hat r_d \\
    \hat r_g \\
    \hat r_{xz} \\
    \hat r_{\tau \kappa}
\end{bmatrix} $$

Where:
\begin{align}
        \hat r_p &:= (1 - \gamma) r_p \\
        \hat r_d &:= (1 - \gamma) r_d \\
        \hat r_g &:= (1 - \gamma) r_g \\ 
        \hat r_{xz} &:= \gamma \mu - xz \\
        \hat r_{\tau \kappa} &:= \gamma \mu - \tau \kappa \\
\end{align}

In [7]:
function get_delta(A, b, c, x, y, z, τ, κ, γ, η)
    m, n = size(A)
    
    rp = b*τ- A*x
    rd = c*τ - A'*y - z
    rg = κ + c'*x - b'*y
    
    μ = (x ⋅ z + τ*κ) / (n + 1)
    
    dinv = x ./ z
    # M is symmetric
    M = A * diagm(dinv) * A'    
    solve = get_solver(M)
    
    α, dx, dz, dtau, dkappa = 0, 0, 0, 0, 0
    dy = missing
    
    for iteration in 1:2
        r⁺p = η(γ) * rp
        r⁺d = η(γ) * rd
        r⁺g = η(γ) * rg

        r⁺xz = γ*μ .- x.*z
        r⁺τκ = γ*μ .- τ*κ
        
        if iteration == 2
            r⁺xz -= dx .* dz
            r⁺τκ -= dtau .* dkappa
        end

        # solve
        p, q = sym_solve(dinv, A, c, b, solve)

        if any(isnan.(p)) || any(isnan.(q))
            throw(DomainError("p, q have NaNs"))
        end

        u, v = sym_solve(dinv, A, r⁺d - (((1) ./ x) .* r⁺xz), r⁺p, solve)

        dtau = (r⁺g + ((1 / τ).* r⁺τκ) - (-c ⋅ u + b ⋅ v)) / (((1) ./ (τ*κ)) + (-c ⋅ p + b ⋅ q))
        dx = u + p .* dtau
        dy = v + q .* dtau
        dz = (1 ./ x) .* (r⁺xz - z .* dx)
        dkappa = 1 ./ τ .* (r⁺τκ - κ .* dtau)
        
        α = get_step(x, dx, z, dz, τ, dtau, κ, dkappa, 1)
        
        β₁ = 0.1
        γ = (1 - α)^2 * min(β₁, 1 - α)
    end
    return dx, dy, dz, dtau, dkappa
end

get_delta (generic function with 1 method)

$$\begin{bmatrix}
    -X^{-1}S & A^T \\
    A & \\
\end{bmatrix} \begin{bmatrix}
    u\\
    v 
\end{bmatrix} = \begin{bmatrix}
    r_1 \\
    r_2
\end{bmatrix}
$$

In [8]:
function sym_solve(dinv, A, r₁, r₂, solve)
    r = r₂ + A * (dinv .* r₁)
    v = solve(r)
    u = dinv .* (A'*v - r₁)
    return u, v
end 

sym_solve (generic function with 1 method)

After the search direction has been computed the variables are updated using
$$(x^+, \tau^+, y^+, s^+, \kappa^+) := (x, \tau, y, s, \kappa) + \alpha (d_x, d_{\tau}, d_y, d_s, d_{\kappa})$$

$\eta = 1 - \gamma$

In [9]:
function do_step(x, y, z, τ, κ, dx, dy, dz, dtau, dkappa, α)
    x = x + α*dx
    y = y + α*dy
    z = z + α*dz
    τ = τ + α*dtau
    κ = κ + α*dkappa
    return x, y, z, τ, κ
end 

do_step (generic function with 1 method)

In [10]:
function get_indicators(A, b, c, x, y, z, τ, κ)
    m, n = size(A)
    x₀, y₀, z₀, τ₀, κ₀ = initial_start(m, n)
    
    rp(x, τ)      = b*τ - A*x
    rd(y, z, τ)   = c*τ - A'*y - z
    rg(x, y, κ)   = κ + c⋅x - b⋅y
    μ(x, τ, z, κ) = (x⋅z + τ*κ) / (n+1)
    
    fx = c ⋅ (x / τ)
    
    rp₀ = rp(x₀, τ₀)
    rd₀ = rd(y₀, z₀, τ₀)
    rg₀ = rg(x₀, y₀, κ₀)
    μ₀  =  μ(x₀, τ₀, z₀, κ₀)
    

    ρp = norm(rp(x, τ)) / max(1, norm(rp₀))
    ρd = norm(rd(y, z, τ)) / max(1, norm(rd₀))
    ρA = norm(c'⋅x - b'⋅y) / (τ + norm(b'⋅y))
    ρg = norm(rg(x, y, κ)) / max(1, norm(rg₀))
    ρμ = μ(x, τ, z, κ) / μ₀
    
    return ρp, ρd, ρA, ρg, ρμ, fx
end
    

get_indicators (generic function with 1 method)

In [11]:
function display_iteration(ρp, ρd, ρg, α, ρμ, fx; header=true)
    if header
        println("Primal Feasibility ",
                "Dual Feasibility   ",
                "Duality Gap        ",
                "Step             ",
                "Path Parameter     ",
                "Objective          ")
    end
    
    s = format("{1:<19.12f}{2:<19.12f}{3:<19.12f}{4:<17.12f}{5:<19.12f}{6:<20.12f}", ρp, ρd, ρg, α, ρμ, fx)
    println(s)
end

display_iteration (generic function with 1 method)

In [12]:
function get_message(status)
    """
    Given problem status code, return a more detailed message.
    Parameters
    ----------
    status : int
        An integer representing the exit status of the optimization::
         0 : Optimization terminated successfully
         1 : Iteration limit reached
         2 : Problem appears to be infeasible
         3 : Problem appears to be unbounded
         4 : Serious numerical difficulties encountered
    Returns
    -------
    message : str
        A string descriptor of the exit status of the optimization.
    """
    messages =
        ["Optimization terminated successfully."
        
         "The iteration limit was reached before the algorithm converged."
        
         "The algorithm terminated successfully and determined that the "
         "problem is infeasible."
        
         "The algorithm terminated successfully and determined that the "
         "problem is unbounded."
         ]
    return messages[status + 1]
end

get_message (generic function with 1 method)

In [13]:
function ip_solve(A, b, c ;α₀=.99995, β=0.1, maxiter=1000, disp=false, tol=1e-8)
    m, n = size(A)
    
    iteration = 0
    status = 0
    
    x, y, z, τ, κ = initial_start(m, n)
    
    ρp, ρd, ρA, ρg, ρμ, fx = get_indicators(A, b, c, x, y, z, τ, κ)
    
    solved = ρp <= tol && ρd <= tol && ρA <= tol
    
    if disp
        display_iteration(ρp, ρd, ρg, NaN, ρμ, fx)
    end
    
    while !solved
        iteration = iteration + 1
        
        γ = 0
        η(γ) = (1 - γ)
            
        # try
        dx, dy, dz, dtau, dkappa = get_delta(A, b, c, x, y, z, τ, κ, γ, η)
    
        α = get_step(x, dx, z, dz, τ, dtau, κ, dkappa, α₀)
        
        x, y, z, τ, κ = do_step(x, y, z, τ, κ, dx, dy, dz, dtau, dkappa, α)
        
        ρp, ρd, ρA, ρg, ρμ, fx = get_indicators(A, b, c, x, y, z, τ, κ)
        
        solved = ρp <= tol && ρd <= tol && ρA <= tol
        
        if disp
            display_iteration(ρp, ρd, ρg, α, ρμ, fx, header=false)
        end
        
        c₁ = ρp < tol && ρd < tol && ρg < tol && τ < tol * max(1, κ)
        c₂ = ρμ < tol && τ < tol * min(1, κ)
        
        if c₁ || c₂
            status = (b' ⋅ y > tol) ? 2 : 3
            break
        elseif iteration >= maxiter
            status = 1
            break
        end
    end
    
    x⁺ = x / τ
    x⁺[x⁺ .< tol] .= 0.0
    z⁺ = z / τ
    y⁺ = y / τ
    return Dict(
        "status"=>status,
        "message"=>get_message(status),
        "x"=>x⁺, 
        "z"=>z⁺, 
        "y"=>y⁺, 
        "nit"=>iteration, 
        "objective"=>c ⋅ x⁺)
end

ip_solve (generic function with 1 method)

In [14]:
function random_problem(m, n)
    A = rand(Float64, (m, m + n))
    x = [4 .+ rand(Float64, m); 1 .+ rand(Float64, n)]
    z = [1 .+ rand(Float64, m); 1 .+ rand(Float64, n)]
    y = rand(Float64, m)
    c = A'*y + z
    b = A*x
    return A, b, c
end

random_problem (generic function with 1 method)

In [15]:
A, b, c = random_problem(200, 200);

In [16]:
@time res = ip_solve(A, b, c, disp=true, tol=1e-8);

Primal Feasibility Dual Feasibility   Duality Gap        Step             Path Parameter     Objective          
1.000000000000     1.000000000000     1.000000000000     NaN              1.000000000000     20533.182959508518  
0.228500551990     0.228500551990     0.228500551990     0.784507232023   0.228500551990     41945.750251814039  
0.082676739047     0.082676739046     0.082665682508     0.654238513156   0.082627866169     52717.113444284478  
0.035016790282     0.035016790282     0.034994981328     0.602559619196   0.034956585985     57478.290834853775  
0.013180204429     0.013180204429     0.013155961574     0.640235005905   0.013109185222     60026.853497157404  
0.004863419666     0.004863419666     0.004850809707     0.649200426658   0.004859384550     61056.837422813092  
0.001315070291     0.001315070291     0.001311475781     0.747532619657   0.001322629761     61498.631492461427  
0.000331392461     0.000331392461     0.000330490255     0.770193058886   0.000334190707 

In [17]:
res

Dict{String,Any} with 7 entries:
  "nit"       => 12
  "status"    => 0
  "message"   => "Optimization terminated successfully."
  "x"         => [7.44223, 2.27532, 1.00084, 5.49962, 0.0, 11.0108, 6.59475, 0.…
  "objective" => 61662.2
  "z"         => [4.38897e-11, 1.18969e-10, 1.96286e-10, 7.95298e-11, 0.611997,…
  "y"         => [0.467225, 0.673809, 0.396829, 0.471738, 0.52464, 0.497452, 0.…

In [18]:
sum(A' * get(res, "y", nothing) + get(res, "z", nothing) - c)

-1.8981253262495557e-6

In [19]:
sum(A * get(res, "x", nothing) - b)

-2.822988392381376e-5

In [20]:
using Cbc
using JuMP

In [21]:
m = Model(Cbc.Optimizer);

In [22]:
_, n = size(A)

(200, 400)

In [23]:
@variable(m, x[1:n]);

In [24]:
@objective(m, Min, c ⋅ x);

In [25]:
@constraint(m, c₁, (A*x) .== b)
@constraint(m, c₂, x .>= 0);

In [26]:
optimize!(m)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 200 (-400) rows, 400 (0) columns and 80000 (-400) elements
0  Obj 0 Primal inf 121144.51 (200)
31  Obj 49016.959 Primal inf 777651.94 (196)
83  Obj 56682.585 Primal inf 101920.43 (162)
129  Obj 58937.939 Primal inf 30327.699 (138)
168  Obj 59761.372 Primal inf 32926.964 (125)
215  Obj 60599.785 Primal inf 14379.659 (118)
254  Obj 61078.229 Primal inf 7252.9636 (103)
295  Obj 61348.048 Primal inf 24775.935 (151)
338  Obj 61477.017 Primal inf 2429.3571 (96)
373  Obj 61569.237 Primal inf 2089.836 (109)
409  Obj 61623.972 Primal inf 20.486258 (77)
451  Obj 61646.552 Primal inf 9.8054744 (78)
495  Obj 61658.68 Primal inf 2.3699585 (49)
526  Obj 61662.207
Optimal - objective value 61662.207
After Postsolve, objective 61662.207, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 61662.20738 - 526 iterations time 0.402, Presolve 0.01
Tot

In [27]:
objective_value(m)

61662.20737937592

In [28]:
sum(A * (value.(x)) - b)

1.88606463780161e-10

In [29]:
sum(A * get(res, "x", 0) - b)

-2.822988392381376e-5

In [30]:
objective_value(m) - get(res, "objective", Inf)

1.4424542314372957e-5

In [31]:
value.(x)

400-element Array{Float64,1}:
  7.442230609514104
  2.2753235737966304
  1.0008351632562793
  5.499615466278851
  0.0
 11.010821838802634
  6.594754547017535
  0.0
  3.5208371484182224
  2.235899626614558
  1.9974193960121958
  8.456785530622705
  0.5434404419111611
  ⋮
  4.914201813889566
  0.0
  1.1767689162003996
  0.0
  5.361576197840911
  0.0
  0.0
  0.0
  1.8614401879567786
  0.9195388418637973
  1.3335566746794496
  0.0